<a href="https://colab.research.google.com/github/AtharvVichare/Plant-Disease-Detector/blob/main/plant_disease_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import csv
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import zipfile
import random
import shutil

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

100%|██████████| 2.70G/2.70G [00:22<00:00, 131MB/s]


Extracting files...


In [ ]:
dataset_path = "/root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2/"
print("Contents of dataset folder:", os.listdir(dataset_path))

Contents of dataset folder: ['new plant diseases dataset(augmented)', 'New Plant Diseases Dataset(Augmented)', 'test']


In [ ]:
dataset_path = "/root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2/new plant diseases dataset(augmented)"

print("Dataset categories:", os.listdir(dataset_path))  # List all class folders


Dataset categories: ['New Plant Diseases Dataset(Augmented)']


In [ ]:
base_dir = "/root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = os.path.join(base_dir, "train")
valid_dir = os.path.join(base_dir, "valid")

In [ ]:
# Define the fraction of data to use (e.g., 20% of total)
subset_fraction = 0.2

# Create new directories for the reduced dataset
small_train_dir = "/content/small_train"
small_valid_dir = "/content/small_valid"

# Copy a fraction of images from the original dataset
def create_subset(original_dir, new_dir, fraction=0.2):
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

    for category in os.listdir(original_dir):
        cat_path = os.path.join(original_dir, category)
        new_cat_path = os.path.join(new_dir, category)

        if not os.path.exists(new_cat_path):
            os.makedirs(new_cat_path)

        images = os.listdir(cat_path)
        subset_images = random.sample(images, int(len(images) * fraction))  # Select fraction

        for img in subset_images:
            src_path = os.path.join(cat_path, img)
            dest_path = os.path.join(new_cat_path, img)
            shutil.copy(src_path, dest_path)

# Apply the subset function
create_subset(train_dir, small_train_dir, subset_fraction)
create_subset(valid_dir, small_valid_dir, subset_fraction)

# Update paths
train_dir = small_train_dir
valid_dir = small_valid_dir


In [ ]:

# Define image size and batch size
img_size = (256, 256)
batch_size = 32

# Image Augmentation for Training Data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescale validation data (No augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load train and validation images in batches
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'  # Since labels are integers
)

val_generator = val_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'
)

# Print class labels mapping
print("Class Indices:", train_generator.class_indices)
num_classes = len(train_generator.class_indices)


Found 14044 images belonging to 38 classes.
Found 3503 images belonging to 38 classes.
Class Indices: {'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15, 'Peach___Bacterial_spot': 16, 'Peach___healthy': 17, 'Pepper,_bell___Bacterial_spot': 18, 'Pepper,_bell___healthy': 19, 'Potato___Early_blight': 20, 'Potato___Late_blight': 21, 'Potato___healthy': 22, 'Raspberry___healthy': 23, 'Soybean___healthy': 24, 'Squash___Powdery_mildew': 25, 'Strawberry___Leaf_scorch': 26, 'Strawberry___healthy

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define Custom CNN
cnn_model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')  # Softmax for multi-class classification
])

# Compile Model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train Model
history_cnn = cnn_model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Save Model
cnn_model.save("cnn_plant_disease_model.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
372/439 ━━━━━━━━━━━━━━━━━━━━ 10:46 10s/step - accuracy: 0.1385 - loss: 4.3973